# Conversation between BlenderBot and Eliza

In this notebook, we create a loop in which Blenderbot has a conversation with Eliza. We send an initial prompt to Blenderbot to start a conversation. We try to extract triples from the Blenderbot response and store these in the eKG.
Next we capture Eliza's response and DialogueGpt's response constinuously until we meet the stop condition. 

In principle, this conversation can go on forever. At the end, we save the scenario in EMISSOR.

Before running, start GraphDB and make sure that there is a sandbox repository.
GraphDB can be downloaded from:

https://graphdb.ontotext.com

## Import the necessary modules

In [1]:
import json
import os
import time
import uuid
from datetime import date
from datetime import datetime
from random import getrandbits, choice
import pathlib
import pprint
import spacy

# general imports for EMISSOR and the BRAIN
import emissor as em
import requests
from cltl import brain
from cltl.brain.long_term_memory import LongTermMemory
from cltl.brain.utils.helper_functions import brain_response_to_json
from cltl.combot.backend.api.discrete import UtteranceType
from cltl.reply_generation.data.sentences import GREETING, ASK_NAME, ELOQUENCE, TALK_TO_ME
from cltl.reply_generation.lenka_replier import LenkaReplier
from cltl.triple_extraction.api import Chat, UtteranceHypothesis
from emissor.persistence import ScenarioStorage
from emissor.representation.annotation import AnnotationType, Token, NER
from emissor.representation.container import Index
from emissor.representation.scenario import Modality, ImageSignal, TextSignal, Mention, Annotation, Scenario

/Users/piek/PycharmProjects/cltl-chatbots/chatvenv/lib/python3.9/site-packages/rdflib_jsonld/__init__.py:9: DeprecationWarning: The rdflib-jsonld package has been integrated into rdflib as of rdflib==6.0.1.  Please remove rdflib-jsonld from your project's dependencies.
  warnings.warn(


In [2]:
#!python -m spacy download en

### Import the chatbot utility functions

In [3]:
import sys
import os

src_path = os.path.abspath(os.path.join('..'))
if src_path not in sys.path:
    sys.path.append(src_path)

#### The next utils are needed for the interaction and creating triples and capsules
import chatbots.util.driver_util as d_util
import chatbots.util.text_util as t_util
import chatbots.util.capsule_util as c_util
import chatbots.intentions.talk as talk
import chatbots.intentions.get_to_know_you as friend
import chatbots.bots.eliza as eliza

## Import a conversation agent pipeline

In [4]:
#from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel, AutoModelWithLMHead
#import torch

#tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-medium')
#model = AutoModelForCausalLM.from_pretrained('microsoft/DialoGPT-medium')

#tokenizer = AutoTokenizer.from_pretrained('gpt2')
#model = AutoModelForCausalLM.from_pretrained('gpt2')

#tokenizer = AutoTokenizer.from_pretrained("manueltonneau/bert-base-cased-conversational-nli")
#model = AutoModel.from_pretrained("manueltonneau/bert-base-cased-conversational-nli")

#tokenizer = AutoTokenizer.from_pretrained("xlnet-large-cased")
#model = AutoModelForCausalLM.from_pretrained("xlnet-large-cased")

#tokenizer = AutoTokenizer.from_pretrained("t5-small")

#model = AutoModelWithLMHead.from_pretrained("t5-small")

#### Needed to suppress messages from DialgGPT
#import os
#os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [4]:
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration
mname = 'facebook/blenderbot-400M-distill'
model = BlenderbotForConditionalGeneration.from_pretrained(mname)
tokenizer = BlenderbotTokenizer.from_pretrained(mname)

In [5]:
UTTERANCE = " I am curious. Has blenderbot live in location?"
inputs = tokenizer([UTTERANCE], return_tensors='pt')
reply_ids = model.generate(**inputs)
print(tokenizer.batch_decode(reply_ids))

["<s> I'm not sure, but I do know that they have been around since the late 19th century.</s>"]


## Standard initialisation of a scenario

We initialise a scenario in the standard way by creating a unique folder and setting the AGENT and HUMAN_NAME and HUMAN_ID variables. Throughout this scenario, the HUMAN_NAME and HUMAN_ID will be used as the source for the utterances.

In [6]:
from random import getrandbits
import requests
##### Setting the location
place_id = getrandbits(8)
location = None
try:
    location = requests.get("https://ipinfo.io").json()
except:
    print("failed to get the IP location")
    
##### Setting the agents
AGENT = "Leolani"
HUMAN_NAME = "BLENDERBOT"
HUMAN_ID = "BLENDERBOT"

### The name of your scenario
scenario_id = datetime.today().strftime("%Y-%m-%d-%H_%M_%S")

### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
# Find the repository root dir
parent, dir_name = (d_util.__file__, "_")
while dir_name and dir_name != "src":
    parent, dir_name = os.path.split(parent)
root_dir = parent
scenario_path = os.path.abspath(os.path.join(root_dir, 'data'))

if not os.path.exists(scenario_path) :
    os.mkdir(scenario_path)
    print("Created a data folder for storing the scenarios", scenario_path)

### Define the folders where the images and rdf triples are saved
imagefolder = scenario_path + "/" + scenario_id + "/" + "image"
rdffolder = scenario_path + "/" + scenario_id + "/" + "rdf"

### Create the scenario folder, the json files and a scenarioStorage and scenario in memory
scenarioStorage = d_util.create_scenario(scenario_path, scenario_id)
scenario_ctrl = scenarioStorage.create_scenario(scenario_id, int(time.time() * 1e3), None, AGENT)

Directories for 2022-05-03-19_43_11 created in /Users/piek/PycharmProjects/cltl-chatbots/data


## Specifying the BRAIN

We specify the BRAIN in GraphDB and use the scenario path just defined for storing the RDF triple produced in EMISSOR.

If you set *clear_all* to *True*, the sandbox triple store is emptied (memory erased) and the basic ontological models are reloaded. Setting it to *False* means you add things to the current memory.

In [7]:
log_path = pathlib.Path(rdffolder)
my_brain = brain.LongTermMemory(address="http://localhost:7200/repositories/blender4",
                                log_dir=log_path,
                                clear_all=True)

2022-05-03 19:44:13,740 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Booted


2022-05-03 19:44:13 -     INFO -                        cltl.brain.basic_brain.LongTermMemory - Booted


2022-05-03 19:44:17,668 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Uploading ontology to brain


2022-05-03 19:44:17 -     INFO -                        cltl.brain.basic_brain.LongTermMemory - Uploading ontology to brain


2022-05-03 19:44:19,216 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Booted


...

2022-05-03 19:44:19 -     INFO -                          cltl.brain.basic_brain.TypeReasoner - Booted


2022-05-03 19:44:19,223 -     INFO -   cltl.brain.basic_brain.TrustCalculator - Booted


2022-05-03 19:44:19 -     INFO -                       cltl.brain.basic_brain.TrustCalculator - Booted


2022-05-03 19:44:19,390 -     INFO -   cltl.brain.basic_brain.TrustCalculator - Computed trust for all known agents


2022-05-03 19:44:19 -     INFO -                       cltl.brain.basic_brain.TrustCalculator - Computed trust for all known agents


## Create an instance of a replier

In [22]:
replier = eliza.ElizaImpl()

## Initialise a chat with the HUMAN_ID to keep track of the dialogue history

In [9]:
chat = Chat(HUMAN_ID)

2022-05-03 19:44:58 -     INFO -                                  cltl.triple_extraction.Chat - << Start of Chat with BLENDERBOT >>


In [10]:
nlp = spacy.load("en_core_web_sm")
#nlp= spacy.load('en') # other languages: de, es, pt, fr, it, nl

In [11]:
from cltl.triple_extraction.cfg_analyzer import CFGAnalyzer
analyzer = CFGAnalyzer()

In [12]:
#CFG test
item = {'utterance': "I like dogs."}
chat.add_utterance(item['utterance'])
analyzer.analyze(chat.last_utterance)
print(chat.last_utterance, chat.last_utterance.triples)

2022-05-03 19:45:08 -     INFO -                                  cltl.triple_extraction.Chat - BLENDERBOT 000: "I like dogs."
2022-05-03 19:45:10 -     INFO -                           cltl.triple_extraction.CFGAnalyzer - Found 1 triples
2022-05-03 19:45:11 -     INFO -              cltl.triple_extraction.GeneralStatementAnalyzer - Utterance type: "STATEMENT"
2022-05-03 19:45:11 -     INFO -              cltl.triple_extraction.GeneralStatementAnalyzer - RDF triplet    subject: {"label": "BLENDERBOT", "type": ["agent"]}
2022-05-03 19:45:11 -     INFO -              cltl.triple_extraction.GeneralStatementAnalyzer - RDF triplet  predicate: {"label": "like", "type": ["emotion"]}
2022-05-03 19:45:11 -     INFO -              cltl.triple_extraction.GeneralStatementAnalyzer - RDF triplet     object: {"label": "dogs.", "type": ["agent"]}
2022-05-03 19:45:11 -     INFO -              cltl.triple_extraction.GeneralStatementAnalyzer - Perspective  certainty: POSSIBLE
2022-05-03 19:45:11 -     IN

BLENDERBOT 000: "I like dogs." [{'object': {'label': 'dogs.', 'type': ['agent']}, 'perspective': {'sentiment': 0.75, 'certainty': 1.0, 'polarity': 1.0, 'emotion': 0.0}, 'predicate': {'label': 'like', 'type': ['emotion']}, 'subject': {'label': 'BLENDERBOT', 'type': ['agent']}, 'utterance_type': 'STATEMENT'}]


## Start the interaction

In [14]:
context_size = 5
def get_answer_from_blender(leolani_prompt:str, history_list:[]):
    answer = ""
    sentences = []
    history = ""
    for i, his in enumerate(history):
        if i==context_size:
            break
        history += his +". "
    input_prompt = history+leolani_prompt
    print('input_prompt:',input_prompt)
    bot_input_ids = tokenizer(input_prompt, return_tensors='pt')
    chat_history_ids = model.generate(**bot_input_ids)
    utteranceList = tokenizer.batch_decode(chat_history_ids)
    print('utteranceList:', utteranceList)
    answer = utteranceList[0].strip('</s>')
    print("ANSWER", answer)
    doc = nlp(answer)
    for s in doc.sents:
        sentence = ""
        for token in s:
            if token.text==',':
                sentences.append(sentence) 
            else:
                sentence += token.text+" "
        sentences.append(sentence) 
    return sentences

In [15]:
def process_text_with_cfg(scenario: Scenario,
                           place_id: str,
                           location: str,
                           human_id: str,
                           textSignal: TextSignal,
                           chat: Chat,
                           analyzer: CFGAnalyzer,
                           my_brain: LongTermMemory,
                           print_details:False):
    capsule = None
    response = None
    response_json = None
    ### Next, we get all possible triples
    chat.add_utterance(c_util.seq_to_text(textSignal.seq))

    analyzer.analyze(chat.last_utterance)

    if print_details:
        print('Last utterance:', c_util.seq_to_text(textSignal.seq))
        print('CFG Tripels:', chat.last_utterance.triples)


    for extracted_triple in chat.last_utterance.triples:
        print(extracted_triple['utterance_type'], extracted_triple)
        capsule = c_util.scenario_utterance_to_capsule(scenario,place_id,location, textSignal,human_id, extracted_triple)
        response = my_brain.update(capsule, reason_types=False, create_label=True)


In [18]:
def process_triple_spacy(scenario: Scenario,
                           place_id: str,
                           location: str,
                           speaker: str,
                           hearer:str,
                           textSignal: TextSignal,
                           my_brain: LongTermMemory,
                           nlp,
                           print_details:False):
    response = None
    triples = t_util.get_subj_amod_triples_with_spacy(textSignal, nlp, speaker, hearer)
    triples.extend(t_util.get_subj_obj_triples_with_spacy(textSignal, nlp, speaker, hearer))
    if print_details:
        print('spacy Triples', triples)

    for triple in triples:
        capsule = c_util.scenario_pred_to_capsule(scenario,
                                                          place_id,
                                                          location,
                                                          textSignal,
                                                          speaker,
                                                          triple[1],
                                                          triple[0],
                                                          triple[2])
        if print_details:
            print('Triple spacy Capsule:')
            pprint.pprint(capsule)

        try:
            response = my_brain.update(capsule, reason_types=False, create_label=True)
            response_json = brain_response_to_json(response)
        except:
            print('Error:', response)

In [ ]:
print_details=True

max_context=50
t1 = datetime.now()
history = []
#### Initial prompt by the system from which we create a TextSignal and store it
leolani_prompt = f"{choice(TALK_TO_ME)}"
history.append(leolani_prompt)
print('\n\t'+AGENT + ": " + leolani_prompt)
textSignal = d_util.create_text_signal_with_speaker_annotation(scenario_ctrl, leolani_prompt, AGENT)
scenario_ctrl.append_signal(textSignal)


#BLENDERBOT

repetition = []
utterance = ""
response_json = None
response_json_list = []
replies = []
#### Get input and loop
#while (datetime.now()-t1).seconds <= 3600 or no_reply_count<15:
    
while True:
    # BLENDER
    answer = ""
    
    sentences = get_answer_from_blender(leolani_prompt, history)
    leolani_prompt = ""

    if not sentences:
        if len(response_json_list)>0:
            for response_json in response_json_list:
                if response_json['statement']:
                    leolani_prompt +=  replier.reply_to_statement(response_json, proactive=True, persist=True)
                response_json_list = []
        else:
            leolani_prompt = f"{choice(TALK_TO_ME)}"

        print('\n\t'+AGENT + ": " + leolani_prompt)
        textSignal = d_util.create_text_signal_with_speaker_annotation(scenario_ctrl, leolani_prompt, AGENT)
        scenario_ctrl.append_signal(textSignal)

    else:
        for utterance in sentences:
            if utterance in repetition:
                print('Repeating', utterance)
                #utterance = None
            else:
                repetition.append(utterance)

            if utterance:
                utterance = utterance.strip()
                if utterance.endswith(' .'):
                    utterance = utterance[:-2]+'.'
                    print('\n\tFixed period '+HUMAN_NAME + ": " + utterance)
                    
                if utterance.endswith(' ?'):
                    utterance = utterance[:-2]+'?'
                    print('\n\tFixed question '+HUMAN_NAME + ": " + utterance)
                    
                textSignal = d_util.create_text_signal_with_speaker_annotation(scenario_ctrl, utterance, HUMAN_ID)
                scenario_ctrl.append_signal(textSignal)

                #### Process input and generate reply

                process_text_with_cfg(scenario_ctrl, place_id, location, HUMAN_ID, textSignal, chat, analyzer, my_brain, print_details)
                process_triple_spacy(scenario_ctrl,place_id,location,HUMAN_ID,AGENT, textSignal,my_brain,nlp,print_details)
                
                leolani_prompt = replier.respond(utterance)
                print('\n\t'+AGENT + ": " + leolani_prompt)
                textSignal = d_util.create_text_signal_with_speaker_annotation(scenario_ctrl, leolani_prompt, AGENT)
                scenario_ctrl.append_signal(textSignal)
                history.append(leolani_prompt)



	Leolani: Would you like to chat? I'll do my best to keep up
input_prompt: Would you like to chat? I'll do my best to keep up


2022-05-03 19:56:46 -     INFO -                                  cltl.triple_extraction.Chat - BLENDERBOT 004: "Sure"


utteranceList: ["<s> Sure, what do you do for a living? I'm an accountant, what about you?</s>"]
ANSWER  Sure, what do you do for a living? I'm an accountant, what about you?


2022-05-03 19:56:47 -  WARNING -                           cltl.triple_extraction.CFGAnalyzer - Couldn't parse input
2022-05-03 19:56:47 -     INFO -                                  cltl.triple_extraction.Chat - BLENDERBOT 005: "Sure what do you do for a living?"


Last utterance: Sure
CFG Tripels: []
spacy Triples []

	Leolani: Please tell me more.

	Fixed question BLENDERBOT: Sure what do you do for a living?


...

2022-05-03 19:59:41,630 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Gaps: 26 gaps as subject: e.g. travel-to location - 15 gaps as object: e.g. like-by agent


2022-05-03 19:59:41 -     INFO -                      cltl.brain.basic_brain.ThoughtGenerator - Gaps: 26 gaps as subject: e.g. travel-to location - 15 gaps as object: e.g. like-by agent


2022-05-03 19:59:41,727 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Gaps: 26 gaps as subject: e.g. work-at institution - 15 gaps as object: e.g. be-friends-with person


2022-05-03 19:59:41 -     INFO -                      cltl.brain.basic_brain.ThoughtGenerator - Gaps: 26 gaps as subject: e.g. work-at institution - 15 gaps as object: e.g. be-friends-with person
2022-05-03 19:59:41 -     INFO -                                  cltl.triple_extraction.Chat - BLENDERBOT 033: "I feel like I need to do something about it but I do n't know what to do."


spacy Triples []

	Leolani: Good, tell me more about these feelings.

	Fixed period BLENDERBOT: I feel like I need to do something about it but I do n't know what to do.


## Save the scenario data

In [36]:
scenario_ctrl.scenario.ruler.end = int(time.time() * 1e3)
scenarioStorage.save_scenario(scenario_ctrl)

## End of notebook